In [1]:
pip install google-generativeai pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import google.generativeai as genai
import json
import os

class FinanceAgent:
    def __init__(self, api_key, csv_path, monthly_budget, current_month=None):
        """
        Initializes the agent.
        """
        # 1. Configure Google AI
        genai.configure(api_key=api_key)
        
        # USE THE STABLE FLASH MODEL
        try:
            self.model = genai.GenerativeModel('gemini-2.0-flash')
        except Exception as e:
            print(f"Error initializing model: {e}")
            print("Falling back to 'gemini-pro'")
            self.model = genai.GenerativeModel('gemini-pro')
        
        self.monthly_budget = monthly_budget
        self.csv_path = csv_path
        
        # 2. Load and Preprocess Data
        try:
            self.df = pd.read_csv(csv_path)
            self.df['Date'] = pd.to_datetime(self.df['Date'])
        except Exception as e:
            raise ValueError(f"Could not read CSV file at {csv_path}. Error: {e}")
        
        # Determine current month (default to latest in data)
        if current_month is None:
            self.current_month = self.df['Date'].max().strftime('%Y-%m')
        else:
            self.current_month = current_month
            
        # Filter data for that month
        self.month_df = self.df[self.df['Date'].dt.strftime('%Y-%m') == self.current_month].copy()
        
        # 3. Auto-Categorize Transactions
        if 'Category' not in self.month_df.columns:
            print("Categorizing transactions with AI (this may take a moment)...")
            self._categorize_transactions()
            
        # 4. Run Initial Analysis
        self._analyze_finances()

    def _categorize_transactions(self):
        """
        Uses the LLM to map unique descriptions to categories.
        """
        unique_descriptions = self.month_df['Description'].unique().tolist()
        
        # Create a prompt to get a JSON mapping
        prompt = f"""
        You are a data processing assistant. Map each of the following transaction descriptions to one of these categories:
        [Housing, Food, Utilities, Entertainment, Transport, Health, Shopping, Travel, Savings, Education, Other].
        
        Descriptions: {unique_descriptions}
        
        Return ONLY a raw JSON object where keys are descriptions and values are categories. Do not use Markdown.
        Example: {{"Starbucks": "Food", "Uber": "Transport"}}
        """
        
        try:
            response = self.model.generate_content(prompt)
            # Clean up potential markdown formatting
            text = response.text.replace("```json", "").replace("```", "").strip()
            category_map = json.loads(text)
            
            # Apply the map
            self.month_df['Category'] = self.month_df['Description'].map(category_map).fillna("Other")
            print("Categorization complete.")
        except Exception as e:
            print(f"Error during categorization: {e}")
            self.month_df['Category'] = "Uncategorized"

    def _analyze_finances(self):
        """
        Calculates hard numbers so the LLM doesn't have to guess math.
        """
        total_spent = self.month_df['Amount'].sum()
        remaining = self.monthly_budget - total_spent
        
        # Spending by Category
        breakdown = self.month_df.groupby('Category')['Amount'].sum().to_dict()
        
        # Store context for the chat
        self.financial_context = {
            "current_month": self.current_month,
            "monthly_budget": self.monthly_budget,
            "total_spent": round(total_spent, 2),
            "remaining_budget": round(remaining, 2),
            "category_breakdown": {k: round(v, 2) for k, v in breakdown.items()}
        }

    def chat(self, user_query):
        """
        The main interface. Takes a user question and answers it using financial context.
        """
        # 1. Build the System Prompt with Context
        system_prompt = f"""
        You are a helpful and fiscally responsible Personal Finance Coach.
        
        CURRENT FINANCIAL STATUS ({self.financial_context['current_month']}):
        - Budget: ${self.financial_context['monthly_budget']}
        - Spent so far: ${self.financial_context['total_spent']}
        - Remaining: ${self.financial_context['remaining_budget']}
        
        SPENDING BY CATEGORY:
        {json.dumps(self.financial_context['category_breakdown'], indent=2)}
        
        INSTRUCTIONS:
        1. Answer the user's question based strictly on the data above.
        2. IF the user asks "Can I afford X?":
           - Compare the cost of X to the 'Remaining' amount.
           - If cost > remaining, say NO and explain why.
           - If cost < remaining, say YES but check if it's a wise purchase.
        3. IF asked for savings advice, look at their highest spending categories.
        4. Be concise and conversational.
        """
        
        full_prompt = f"{system_prompt}\n\nUser: {user_query}\nAgent:"
        
        try:
            response = self.model.generate_content(full_prompt)
            return response.text
        except Exception as e:
            return f"Error generating response: {e}"

# --- MAIN EXECUTION ---
MY_API_KEY = "my_api_key"

# 2. Initialize Agent
if MY_API_KEY == "YOUR_GOOGLE_API_KEY_HERE":
    print("⚠️ ERROR: You must paste your Google API Key in the 'MY_API_KEY' variable on line 118.")
else:
    print("Initializing Agent...")
    try:
        agent = FinanceAgent(
            api_key=MY_API_KEY, 
            csv_path='transactions.csv', 
            monthly_budget=50000 
        )

        print(f"\n--- Finance Agent Ready (Month: {agent.financial_context['current_month']}) ---")
        print(f"Remaining Budget: ${agent.financial_context['remaining_budget']}")
        print("Type 'exit' to stop.")

        # 3. Interactive Loop
        while True:
            user_input = input("\nYou: ")
            if user_input.lower() in ['exit', 'quit']:
                break
            
            response = agent.chat(user_input)
            print(f"Agent: {response}")

    except Exception as e:
        print(f"Failed to initialize agent: {e}")

Initializing Agent...
Categorizing transactions with AI (this may take a moment)...
Categorization complete.

--- Finance Agent Ready (Month: 2024-12) ---
Remaining Budget: $31972.62
Type 'exit' to stop.
Agent: Okay, let's see about that iPhone 17. I can't tell you if you can *afford* it without knowing how much it costs! I also can't tell you how much an iPhone 17 is because it does not exist yet. 

However, based on your remaining budget of $31,972.62, you *probably* can.

But before you rush out to buy one, let's think about whether it's a *wise* purchase. Your highest spending categories are Shopping, Transport, and Entertainment. Maybe consider if there are ways to cut back in those areas first before splurging on a new phone. Is your current phone really not working for you?

Agent: Based on your current financial status, YES, you *can* technically afford a brand new Honda Civic, as your remaining budget ($31,972.62) is likely more than the cost of the car. However, before you ma